<a href="https://colab.research.google.com/github/young-hwanlee/gnn_example/blob/main/gnn_basics_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/young-hwanlee/gnn_example.git

fatal: destination path 'gnn_example' already exists and is not an empty directory.


In [2]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install torch_geometric
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

2.3.1+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 39.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


# **PyG Basics**

In [3]:
import torch

x = torch.tensor([[-1], [0], [1]], dtype=torch.float)
edge_index = torch.tensor([[0, 1, 1, 2],    # 출발 노드 인덱스
                                             [1, 0, 2, 1]],   # 도착 노드 인덱스
                          dtype=torch.long)

In [4]:
print(x)
print(edge_index)

tensor([[-1.],
        [ 0.],
        [ 1.]])
tensor([[0, 1, 1, 2],
        [1, 0, 2, 1]])


In [5]:
from torch_geometric.data import Data

data = Data(x=x, edge_index=edge_index)
print(data)

Data(x=[3, 1], edge_index=[2, 4])


In [6]:
data.num_nodes

3

In [7]:
data.num_edges

4

In [8]:
data.has_isolated_nodes()

False

In [9]:
data.has_self_loops()

False

In [10]:
data.is_directed()

False

In [11]:
# import torch
# from torch_geometric.data import Data

x = torch.tensor([[-1], [0], [1]], dtype=torch.float)
edge_index = torch.tensor([[0, 1],
                        [1, 0],
                        [1, 2],
                        [2, 1]], dtype=torch.long)

data = Data(x=x, edge_index=edge_index.t().contiguous())
print(data)

Data(x=[3, 1], edge_index=[2, 4])


# **Dataset Example 1**

In [12]:
!pip install fsspec==2024.3.1

In [13]:
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root='/tmp/ENZYMES/', name='ENZYMES')
print(dataset)

ENZYMES(600)


In [14]:
len(dataset)

600

In [15]:
dataset.num_classes

6

In [16]:
dataset.num_node_features

3

In [17]:
data = dataset[0]
print(data)

Data(edge_index=[2, 168], x=[37, 3], y=[1])


In [18]:
data.is_undirected()

True

In [19]:
train_dataset = dataset[:540]
print(train_dataset)

ENZYMES(540)


In [20]:
test_dataset = dataset[540:]
print(test_dataset)

ENZYMES(60)


# **Dataset Example 2**

In [21]:
from torch_geometric.datasets import planetoid

dataset = planetoid.Planetoid(root='/tmp/Cora', name='Cora')
print(dataset)

Cora()


In [22]:
len(dataset)

1

In [23]:
dataset.num_classes

7

In [24]:
dataset.num_node_features

1433

In [25]:
data = dataset[0]
print(data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [26]:
data.is_undirected()

True

In [27]:
data.train_mask.sum().item()

140

In [28]:
data.val_mask.sum().item()

500

In [29]:
data.test_mask.sum().item()

1000

# **Mini-Batch Loader**

In [30]:
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader

dataset = TUDataset(root='/tmp/ENZYMES/', name='ENZYMES', use_node_attr=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [31]:
print(loader)

In [32]:
for batch in loader:
  print(batch)
  print(batch.num_graphs)

DataBatch(edge_index=[2, 3978], x=[988, 21], y=[32], batch=[988], ptr=[33])
32
DataBatch(edge_index=[2, 3914], x=[1029, 21], y=[32], batch=[1029], ptr=[33])
32
DataBatch(edge_index=[2, 3840], x=[1117, 21], y=[32], batch=[1117], ptr=[33])
32
DataBatch(edge_index=[2, 3604], x=[990, 21], y=[32], batch=[990], ptr=[33])
32
DataBatch(edge_index=[2, 4212], x=[1092, 21], y=[32], batch=[1092], ptr=[33])
32
DataBatch(edge_index=[2, 3832], x=[1005, 21], y=[32], batch=[1005], ptr=[33])
32
DataBatch(edge_index=[2, 3840], x=[987, 21], y=[32], batch=[987], ptr=[33])
32
DataBatch(edge_index=[2, 3386], x=[919, 21], y=[32], batch=[919], ptr=[33])
32
DataBatch(edge_index=[2, 3922], x=[1009, 21], y=[32], batch=[1009], ptr=[33])
32
DataBatch(edge_index=[2, 4156], x=[1054, 21], y=[32], batch=[1054], ptr=[33])
32
DataBatch(edge_index=[2, 4286], x=[1104, 21], y=[32], batch=[1104], ptr=[33])
32
DataBatch(edge_index=[2, 4146], x=[1063, 21], y=[32], batch=[1063], ptr=[33])
32
DataBatch(edge_index=[2, 4356], x=[1

# **Data Transform**

In [33]:
## If the error (BadZipFile: File is not a zip file) occurs, simply delete the "ShapeNet" folder and run the code.
from torch_geometric.datasets import ShapeNet

dataset = ShapeNet(root='/tmp/ShapeNet', categories=['Airplane'], force_reload=True)
print(dataset[0])

Processing...


Data(x=[2518, 3], y=[2518], pos=[2518, 3], category=[1])


Done!


In [34]:
import torch_geometric.transforms as T
from torch_geometric.datasets import ShapeNet

dataset = ShapeNet(root='/tmp/ShapeNet', categories=['Airplane'],
                   pre_transform=T.KNNGraph(k=6),
                   transform=T.RandomJitter(0.01), force_reload=True)
print(dataset[0])

/usr/local/lib/python3.10/dist-packages/torch_geometric/data/dataset.py:239: UserWarning: The `pre_transform` argument differs from the one used in the pre-processed version of this dataset. If you want to make use of another pre-processing technique, pass `force_reload=True` explicitly to reload the dataset.
  warnings.warn(
Processing...
Done!


Data(x=[2518, 3], y=[2518], pos=[2518, 3], category=[1], edge_index=[2, 15108])
